# <center> **Lab 2 Extra : Comparing Bag of Words to Spatial Pyramid Matching** <center>

Both BoW and SPM algorithms are based on the same concept, extracting features from an image by dividing it into grids, grouping the feature using K-means clustering and finding the main features by analysing the feature histogram. However, SPM is better than BoW because it has spatial awareness, that is, it can locate which feature is at which part of the image.  


<br>


### Bow and SPM Algorithm



1. The image is first divided into a grid into different blocks
2. For each block, SIFT features are extracted
3. These SIFT features are all a bit different and these slightly diff features need to be clustered together to be seen as one feature. This grouping is done using K means clustering 
4. After grouping the features, a histogram is created with x axis as features and y axis as number of blocks. For BoW, only one such histogram is made. However, for SPM, image is first divided into sub-parts, and number of features in each sub part are again counted, and histograms are again computer for each sub part
5. Looking at the histogram and which feature has most blocks we can say what object is present. 

<br>

**Since BoW has only one histogram that represents all the features, it does not have any spatial awareness. Since SPM has several histograms, each consisting of a number of features corresponding to a particular region, it has spatial awareness.**

<br>


> **Note** : SIFT (Scale-Invariant Feature Transform)is an algorithm used to detect and describe local features in digital images. It locates certain key points and then furnishes them with quantitative information (so-called descriptors) which can for example be used for object recognition.





## Loading and pre-processing data

In [ ]:
# LOADING DEPENDENCIES
import cv2
import numpy as np
import glob
import copy
import math

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.svm import LinearSVC

import splitfolders

In [ ]:
# DEFINING FUNCTION TO GET THE DATASET
def get_data(path, num_per_class=-1):
  data = []
  labels = []
  for id, class_name in class_names.items():
    img_path_class = glob.glob(path + class_name + '/*.jpg')
    if num_per_class > 0:
        img_path_class = img_path_class[:num_per_class]
    labels.extend([id]*len(img_path_class))
    for filename in img_path_class:
        data.append(cv2.imread(filename, 0))
  return data, labels

In [ ]:
# SAVING THE CLASSIFICATION CATEGORY NAMES
class_names = [name[21:] for name in glob.glob('101_ObjectCategories/*')]
class_names = dict(zip(range(0,len(class_names)), class_names))

# SPLITTING DATASET INTO 70:30 RATIO
splitfolders.ratio('101_ObjectCategories/', output="101_split/", seed=1337, ratio=(.7, .3)) # default values

# LOADING TRAINING DATA
train_data, train_label = get_data('101_split/train/', 100)
train_num = len(train_label)

# LOADING TESTING DATA
test_data, test_label = get_data('101_split/val/', 100)
test_num = len(test_label)

## Implementing Bag of Features

Step 1 and 2 : Dividing the dataset into blocks and getting the different features using SIFT



In [ ]:
#FUNCTION TO COMPUTE SIFT DESCRIPTORS
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 15 # NO OF GRIDS 
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        x.append(dense_feat[1])     
    return x

In [ ]:
# USING SIFT TO EXTRACT FEATURES FROM TRAINING IMAGES

x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)

all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)
print(x_train)

[array([[  0.,   0.,   0., ...,  26., 174., 150.],
       [  0.,   0.,   0., ...,  24., 154.,  99.],
       [  0.,   0.,   0., ...,  24., 141.,  63.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32), array([[  0.,   0.,   0., ..., 127.,  41.,  21.],
       [  0.,   0.,   0., ...,  98.,  17.,  13.],
       [  0.,   0.,   0., ...,  92.,  21.,  15.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32), array([[  0.,   0.,   0., ...,  30.,  90., 128.],
       [  0.,   0.,   0., ...,  21.,  60., 117.],
       [  0.,   0.,   0., ...,  25.,  59., 112.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]], dtype=float32), array([[  0.

Step 3 and 4 : Grouping using K-means clustering and drawing histograms

In [ ]:
# build BoW presentation from SIFT of training images 
def clusterFeatures(all_train_desc, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(all_train_desc)
    return kmeans


# form training set histograms for each training image using BoW representations
def formHistogram(x_train, kmeans, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict = kmeans.predict(data)
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)
    

def accuracy(predict_label, test_label):
    return np.mean(np.array(predict_label.tolist()[0]) == np.array(test_label))

In [ ]:
k = 60
print(all_train_desc)
kmeans = clusterFeatures(all_train_desc, k)

# form training and testing histograms
train_hist = formHistogram(x_train, kmeans, k)
test_hist = formHistogram(x_test, kmeans, k)

# normalize histograms
scaler = preprocessing.StandardScaler().fit(train_hist)
train_hist = scaler.transform(train_hist)
test_hist = scaler.transform(test_hist)

# Using Support Vector Machines (SVMs) to train our model
clf = LinearSVC(random_state=0, C=1.0)
clf.fit(train_hist, train_label)
predict = clf.predict(test_hist)
print("C =", 1.0, ",\t Accuracy:", np.mean(predict == test_label)*100, "%")

[[  0.   0.   0. ...  26. 174. 150.]
 [  0.   0.   0. ...  24. 154.  99.]
 [  0.   0.   0. ...  24. 141.  63.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
C = 1.0 ,	 Accuracy: 44.00166044001661 %


**Thus, an accuracy if 44% approx was obtained using BoW**


## Implementing Spacial Pyramid Matching

In [ ]:
# Step 1 and 2 : Dividing data into grids and extracting features using SIFT
def extract_denseSIFT(img):
    DSIFT_STEP_SIZE = 2
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]

    descriptors = sift.compute(img, keypoints)[1]
    
    return descriptors


# Step 3 and 4 : Using KMeans to form groups of features and using these features to form histogram 
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])                

                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist


# Get histogram representation for training/testing data
def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in range(len(data)):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [ ]:
train_histo = getHistogramSPM(2, train_data, kmeans, k)
test_histo = getHistogramSPM(2, test_data, kmeans, k)

# train SVM
clf = LinearSVC(random_state=0, C=0.01)
clf.fit(train_histo, train_label)
predict = clf.predict(test_histo)
print("C =", 0.01, ",\t\t Accuracy:", np.mean(predict == test_label)*100, "%")

C = 0.01 ,		 Accuracy: 59.65130759651308 %


**Thus, an accuracy of 59.65% approx was obtained using SPM**


## Conclusion

As mentioned before, BoW does not take into consideration the spatial information of the image, as opposed to SPM. This is why SPM tends to have higher classification accuracy than BoW. We can see here that we are getting a classification accuracy of 44% by using BoW, and 59.61% by using SPM.
